In [1]:
import sys
import os

In [2]:
sys.path.append(os.path.abspath('..'))

In [3]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.datasets import make_circles

# Tensor class
from pynet.tensor import Tensor

# Neural network modules
from pynet.nn.abstract import Module
from pynet.nn.sequential import Sequential
from pynet.nn.linear import Linear
from pynet.nn.relu import ReLU
from pynet.nn.sigmoid import Sigmoid

# Datasets
from pynet.data.in_memory import InMemoryDataset

# Loss functions
from pynet.loss.bce import BinaryCrossEntropy

# Optimizers
from pynet.optimizers.sgd import SGD

# Trainer and training/testing callbacks
from pynet.training.trainer import Trainer
from pynet.training.callbacks.abstract import Callback
from pynet.training.callbacks.print import PrintCallback
from pynet.training.history import History

In [11]:
class GifCallback(Callback):
    def __init__(self, model: Module, x: np.ndarray, y: np.ndarray, imgdir: str) -> None:
        super().__init__()

        self.__model = model
        self.__x = x
        self.__y = y
        self.__imgdir = imgdir
    
    def on_epoch_end(self, history: History) -> None:
        model = self.__model
        x = self.__x
        y = self.__y
        epoch = history.rows[-1].epoch

        x_min, x_max = x[:, 0, 0].min() - 0.1, x[:, 0, 0].max() + 0.1
        y_min, y_max = x[:, 1, 0].min() - 0.1, x[:, 1, 0].max() + 0.1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

        grid = np.c_[xx.ravel(), yy.ravel()]
        grid = np.expand_dims(grid, axis=2)

        preds = [model.forward(Tensor(grid[i])) for i in range(len(grid))]
        z = np.array([round(pred.ndarray.item()) for pred in preds])
        z = z.reshape(xx.shape)

        fig, ax = plt.subplots(figsize=(10, 10))
        c_dict = {0: "tab:green", 1: "tab:red"}
        colors =list(np.vectorize(c_dict.get)(y))

        ax.contourf(xx, yy, z, 1, alpha=0.4, colors=["tab:green", "tab:red"])
        ax.scatter(x[:,0,0], x[:,1,0], c=colors)
        ax.set_title(f"Epoch: {epoch + 1}")

        os.makedirs(self.__imgdir, exist_ok=True) 

        plt.savefig(os.path.join(self.__imgdir, f"epoch_{epoch}.jpg"), format="jpg")
        plt.close(fig)

In [12]:
x, y = make_circles(1000, noise=0.025)
# inputs to neural net must be of shape [n, 1]
x = np.expand_dims(x, axis=2)

model = Sequential([
    Linear(2, 16),
    ReLU(),
    Linear(16, 1),
    Sigmoid()
])

dataset = InMemoryDataset(x, y)
loss_f = BinaryCrossEntropy()
sgd = SGD(0.01, 0.9)
callbacks = [PrintCallback(), GifCallback(model, x, y, ".\\training_visualization")]
trainer = Trainer()

In [13]:
trainer.train(model, dataset, None, loss_f, sgd, 30, callbacks)

Epoch 0001 -> train_loss: 0.7050, train_accuracy: 0.5160
Epoch 0002 -> train_loss: 0.6687, train_accuracy: 0.6030
Epoch 0003 -> train_loss: 0.6434, train_accuracy: 0.6690
Epoch 0004 -> train_loss: 0.6179, train_accuracy: 0.7400
Epoch 0005 -> train_loss: 0.5917, train_accuracy: 0.8080
Epoch 0006 -> train_loss: 0.5629, train_accuracy: 0.8610
Epoch 0007 -> train_loss: 0.5332, train_accuracy: 0.9240
Epoch 0008 -> train_loss: 0.4995, train_accuracy: 0.9530
Epoch 0009 -> train_loss: 0.4675, train_accuracy: 0.9700
Epoch 0010 -> train_loss: 0.4332, train_accuracy: 0.9720
Epoch 0011 -> train_loss: 0.4013, train_accuracy: 0.9920
Epoch 0012 -> train_loss: 0.3669, train_accuracy: 0.9920
Epoch 0013 -> train_loss: 0.3357, train_accuracy: 0.9910
Epoch 0014 -> train_loss: 0.3039, train_accuracy: 0.9910
Epoch 0015 -> train_loss: 0.2767, train_accuracy: 0.9950
Epoch 0016 -> train_loss: 0.2525, train_accuracy: 0.9930
Epoch 0017 -> train_loss: 0.2298, train_accuracy: 0.9980
Epoch 0018 -> train_loss: 0.207

In [16]:
img, *imgs = [Image.open(f"training_visualization\\epoch_{i}.jpg") for i in range(20)]
img.save("training_visualization\\training.gif", format="GIF", append_images=imgs, save_all=True, duration=500, loop=0)